In [1]:
import pandas as pd

movies_df = pd.read_csv('ml-latest-small/movies.csv')
ratings_df = pd.read_csv('ml-latest-small/ratings.csv')

In [7]:
# Sparse user-item matrix. 

user_item_matrix = ratings_df.pivot_table(index='userId', columns='movieId', values='rating')

print(user_item_matrix.head())

movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     NaN     4.0     NaN     NaN     4.0     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           4.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  9       10      ...  193565  193567  193571  193573  193579  193581  \
userId                   ...                                                   
1           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     N

In [8]:
# Counting the total number of non-NaNs and NaNs in the user-item matrix
non_nan_count = user_item_matrix.notna().sum().sum()
nan_count = user_item_matrix.isna().sum().sum()

print("Number of non-NaN values in the matrix:", non_nan_count)
print("Number of NaN values in the matrix:", nan_count)

Number of non-NaN values in the matrix: 100836
Number of NaN values in the matrix: 5830804


In [ ]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)  # 70% train, 30% temp
